In [9]:
from sentiment_analysis_model import imdb_sentiment_analysis
import numpy as np
imdb = imdb_sentiment_analysis()

raw_X_train, y_train= imdb.load_train_data(imdb.negative_folder, imdb.positive_folder)
raw_X_test = imdb.load_test_data(imdb.test_folder)
all_X = raw_X_train + raw_X_test

NameError: name 'imdb' is not defined

Import the appropriate module from our own python script, then load the raw training and test data.

# Bag of Words Pre-Processing
If you wish to preprocess data with the bag of words approach, use our preprocess() function. It's function that takes as input optional boolean parameters that represent the type of preprocessing you wish to perform on the data:

binary_text_features: boolean - if true will append to the input data a count vectors for each words through each comment, with only two counts: 1 (if the word appears in the comment, 0 otherwise)

multi_text_features: boolean - if true will append to the input data multinomial count features

num_features: int default 10 - the number of "common" words to count vectorize for each comment.

bi_grams: boolean - if true, will append to the input data bi_gram count vectorizers

num_bi_grams: int default 10- - same as num_features but for bi_grams

tri_grams: boolean- same as bi_grams

num_tri_grams: int default 10, same as num_features, num_bi_grams

tf_idf:  if true, will apply a tfidf transform to all the input word features specified by the other parameters above.


In [10]:
x_train_BOW, y_BOW, x_test_BOW, shuffled_raw_X_train = imdb.preprocess(all_X, y_train, binary_text_features=True , num_features=1000)


NameError: name 'all_X' is not defined

In the line of code above, we are using the 1000 most common words in the imdb movie review corpus and checking if they're in each comment (1) or if they aren't (0). Note that all the transformations performed on the training set are also performed on the test set. Also note, this can take a WHILE

# Feature Selection
In our best best model, we used top 60 percent using f_classif, then top 55 percent using chi2. It is recommended that you save the default values of x_train_BOW etc somewhere else so that they don't take FOREVER to reload via the preprocess function above.


In [11]:
from sklearn.feature_selection import SelectPercentile, chi2, f_classif, mutual_info_classif
features = SelectPercentile(f_classif, percentile=60)
features = SelectPercentile(chi2, percentile=55)
x_train_BOW = features.fit_transform(x_train_BOW, y_train_BOW)
x_val_BOW = features.transform(x_val_BOW)
x_test_BOW = features.transform(x_test_BOW)


NameError: name 'x_train_BOW' is not defined

# Classifiers
Now the fun part.

## Run Bernoulli Naive Bayes:
Run our homemade naive bayes:

In [12]:
bnm_classifier = imdb.run_standard_classification(x_train_BOW, y_BOW, naive_bayes=True)

NameError: name 'x_train_BOW' is not defined

As you've noticed this is a little slow. For testing and development therefore we use the sklearn version.

In [ ]:
from sklearn.naive_bayes import BernoulliNB
classifier = BernoulliNB()
classifier = classifier.fit(x_train_BOW, y_BOW)
score_bnb = cross_val_score(classifier, x_train_BOW, y_BOW, cv=5, scoring='accuracy')
print("Bernoulli Naive Bayes Accuracy : "  + str(np.mean(score_bnb)))

## Run MultiVariate Naive Bayes:

In [ ]:
multi_bayes_classifier = imdb.multiVar_NB(x_train_BOW, y_BOW)
y_pred_bayes_class = multi_bayes_classifier.predict(x_val_BOW)
acc_bayes = accuracy_score(y_pred_bayes_class, y_val_BOW)
score_mnb = cross_val_score(multi_bayes_classifier, x_train_BOW, y_BOW, cv=5, scoring='accuracy')
print("MultiVariate Naive Bayes Accuracy : " + str(np.mean(score_mnb)))

## Run Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rand_forest_classifier=RandomForestClassifier(n_estimators=50)
score_rand_for = cross_val_score(rand_forest_classifier, x_train_BOW, y_BOW, cv=5, scoring='accuracy')
print("Random Forest Accuracy : " + str(np.mean(score_rand_for)))

## Run Logistic Regression Model

In [ ]:
logreg_classifier = imdb._logistic_regression_model(x_train_BOW, y_BOW)
score_logreg = cross_val_score(logreg_classifier, x_train_BOW, y_BOW, cv=5, scoring='accuracy')
print("Logistic Regression Classifier Accuracy: " + str(np.mean(score_logreg)))

## Run Support Vector Machine Model

In [ ]:
svm_classifier = imdb.support_vector_machine(x_train_BOW, y_BOW)
score_svm = cross_val_score(svm_classifier, x_train_BOW, y_BOW, cv=5, scoring='accuracy')
print("Support Vector Machine Classifier Accuracy : " + str(np.mean(score_logreg)))

# Preprocessing for CNN

The following is for preprocessing for the CNN. Warning, this takes a long long time to train. 
preprocess the data into vectors. Do not use any of the boolean parameters above for the preprocessing, stick just to the vector and the num_features.

In [ ]:
all_X=shuffled_raw_X_train + raw_X_test
x_train_vec, y_vec, x_test_vec, shuffled_raw_X_train = imdb.preprocess(all_X, y_BOW, vector=True, num_features=10000)

x_train_vec, x_val_vec = [x_train_vec[:20000], x_train_vec[20000:]]
y_train_vec, y_val_vec = [y_vec[:20000], y_vec[20000:]]


Train the classifier using the same number of features as inputed in the preprocessing.

In [ ]:
neural_classifier = imdb.neural_1DCNN_2(x_train_vec, y_train_vec, 100000 )

y_pred_1 = neural_classifier.predict_classes(x_val_vec)
acc_1 = accuracy_score(y_val_vec, y_pred_1)

print("neural_classifier accuracy: " + str(acc_1))